# Práctica 2: Algoritmos metaheurísticos

## Sistemas Inteligentes

### Curso académico 2022-2023

#### Profesorado:

* Juan Carlos Alfaro Jiménez (`JuanCarlos.Alfaro@uclm.es`)
* Guillermo Tomás Fernández Martín (`Guillermo.Fernandez@uclm.es`)
* María Julia Flores Gallego (`Julia.Flores@uclm.es`)
* Ismael García Varea (`Ismael.Garcia@uclm.es`)
* Luis González Naharro (`Luis.GNaharro@uclm.es`)
* Aurora Macías Ojeda (`Profesor.AMacias@uclm.es`)
* Marina Sokolova Sokolova (`Marina.Sokolova@uclm.es`)

<br>
<div style="text-align: left">
<font color="#4E70BE" size=3>Integrantes Grupo 7:</font><br>
<ul>
<li><font color="#4E70BE" size=3>Daniel Cabañero Pardo</font><br></li>
<li><font color="#4E70BE" size=3>Pedro Jesús Martínez Herrero</font><br></li>
 
</ul>

## 0. Preliminares

Antes de comenzar con el desarrollo de esta práctica es necesario **descargar**, en **formato `.py`**, el código de la **práctica anterior** con el nombre **`utils.py`**. Para ello, pulsamos, en la libreta de la primera práctica, **`File > Export .py`**.

Una vez hemos descargado y nombrado correctamente el fichero, lo **añadimos** al espacio de trabajo de la **libreta** de la práctica **actual** a través de **`Attached data > Notebook files > Upload files`** y subimos el fichero `utils.py` descargado en el paso anterior.

Tras esto, debemos **cambiar** el **constructor** de la clase **`Problem`** para que **reciba** directamente el **problema** a resolver, **en lugar de cargarlo desde** un **fichero**. Esto se debe a que los algoritmos metaheurísticos van a tener que resolver, en múltiples ocasiones, este problema. De esta manera nos **evitamos** la **carga computacional extra** que implica **leer** el problema desde un **fichero**. Además, también es necesario **comentar** cualquier línea de **código** que **imprima estadísticas** para evitar salidas largas.

## 1. Introducción

En esta práctica, vamos a **resolver** un **problema** de **optimización combinatoria** mediante **algoritmos metaheurísticos**. En particular, vamos a implementar **algoritmos genéticos** para abordar el **problema** de **ruteo** de **vehículos**. En este, **varios vehículos** con **capacidad limitada** deben **recoger paquetes** en **diferentes ubicaciones** y **trasladarlos** a una **sede central** de recogida.

Además, se **analizará** y **comparará** el **rendimiento** de **diferentes algoritmos genéticos** (mediante la modificación de los pasos correspondientes) en diferentes instancias del problema.

---

## 2. Descripción del problema

El concepto de mapa que vamos a utilizar es similar al de la primera práctica. Este se representa mediante un **grafo**, donde los **nodos** representan **ciudades** y los **enlaces** indican la existencia de una **carretera en ambos sentidos** entre dos ciudades. Además, los **enlaces** tienen un **peso** asociado indicando la **distancia real** entre las dos ciudades. Al mismo tiempo, se proporciona una **sede central**, que se trata de la ciudad donde se deben dejar los paquetes. A su vez, se dispone de una **flota de vehículos** con **capacidad limitada** que deben **recoger** los **paquetes** en las ciudades correspondientes y que **inicialmente** están aparcados en la **sede central**. **En caso de que a la hora de recoger un paquete se supere la capacidad del vehículo correspondiente, este debe volver a la sede central a descargar todos los paquetes, considerando el coste que implicaría volver**.

Un mapa es un problema en particular, pero diferentes paquetes, capacidades de vehículos y ubicación de la sede central pueden dar lugar a diferentes instancias del problema. Por tanto, el **objetivo** en este problema es **recoger todos los paquetes de tal manera que los vehículos recorran la menor distancia posible**.

**Con el objetivo de simplificar la práctica en evaluación continua, se asume que se cuenta con un único vehículo. No obstante, esto podría cambiar para la evaluación no continua.**

---

## 3. Desarrollo de la práctica

Durante el desarrollo de la práctica se va a proporcionar un conjunto de mapas, sobre los cuáles se debe resolver el problema de optimización combinatoria correspondiente. Es importante destacar que la **dimensionalidad** del **problema** (número de ciudades, carreteras y paquetes) puede ser **variable**, por lo que los diferentes **algoritmos genéticos** deben ser lo suficientemente **eficientes** para que puedan **resolver** los **problemas** en un **tiempo razonable**.

**Además, algunos escenarios se van a guardar para las entrevistas de prácticas, por lo que el código debe ser lo más general posible para cargarlos de manera rápida y sencilla.**

### 3.1. Entrada

Cada escenario tendrá un fichero `.json` asociado con la siguiente estructura:

```JSON
{
    "map": {
        "cities": [
            {
                "id": id_city_0,
                "name": name_city_0,
                "lat": latitude_city_0,
                "lon": longitude_city_0
            }
        ],
        "roads": [
            {
                "origin": origin_city_id,
                "destination": destination_city_id,
                "distance": road_distance
            }
        ]
    },
    "warehouse": warehouse_city_id,
    "vehicles": [
        {
            "id": id_vehicle_0,
            "capacity": capacity_vehicle_0
        }
    ]
    "parcels": [
        {
            "id": id_parcel_0,
            "city": parcel_city_id,
            "weight": weight_parcel_0
        }
    ]
}
```

Hay cuatro elementos principales en el fichero:

* `map`: Un diccionario con el mapa, cuya descripción es la misma que la de la primera práctica
* `warehouse`: Identificador de la ciudad donde se encuentra la sede central
* `vehicles`: Lista de vehículos disponibles
* `parcels`: Lista de paquetes a recoger

Por su parte, `vehicles` contiene:

* `id`: Identificador del vehículo
* `capacity`: Capacidad máxima del vehículo

Y `parcels`:

* `id`: Identificador del paquete
* `city`: Ciudad donde se encuentra el paquete
* `weight`: Peso del paquete

**Para añadir los ficheros con los problemas al espacio de trabajo se debe usar el mismo procedimiento anterior.**

---

## 4. Plan de trabajo

### 4.1. Formalización del problema

Para resolver cualquier problema de optimización combinatoria en primer lugar hay que definir como vamos a **codificar** las **soluciones** al problema. Si bien es algo que se deja a criterio propio, se plantea la siguiente pregunta, **¿cuál puede ser la mejor representación para una secuencia de paquetes a recoger?**

Se puede comprobar si la respuesta es correcta introduciéndola en la variable `answer` del siguiente fragmento de código:

In [24]:
# Third party
import hashlib

In [25]:
check_answer = lambda answer, hashed: "The answer is " + ("" if hashlib.md5(answer).hexdigest() == hashed else "in") + "correct."

In [26]:
# TODO: Introduce here the answer to use for the hashing
answer = "permutation"

# Avoid case sensitivity in the answer
answer = str.lower(answer)

# Encode the answer before hashing
answer = answer.encode("utf-8")

hashed = "90d377b31e1ac26d0d10d5612ce33ccc"  # The hashed answer
print(hashed)

check_answer(answer, hashed)

90d377b31e1ac26d0d10d5612ce33ccc


'The answer is correct.'

### 4.2. Implementación

A continuación se proporciona la estructura de clases recomendada para resolver el problema en cuestión. Tendréis que completar las siguientes clases de acuerdo con los algoritmos estudiados en teoría. **Debéis incluir en la siguiente celda todas las librerías que vayáis a utilizar para mantener la libreta lo más organizada posible**:

In [27]:
# TODO: Import here the libraries
import utils
from utils import *
import math
import numpy as np

#### Clase `Individual`

Esta clase proporciona la **codificación** de un **individuo** de la **población**.

Los **métodos obligatorios** que se deben añadir son:

* ``__init__(self, num_genes, generation_type, crossover_type, mutation_type)``: Inicializa el **número** de **genes** del **individuo** y el **tipo** de **operación** de **generación**, **cruce** y **mutación**. Ademas, genera la **solución** que **representa** el **individuo**.
* ``generate(num_genes, generation_type)``: Método estático para **generar** una **solución** del tamaño proporcionado de acuerdo con el tipo de operación de generación.
* ``crossover(self, individual)``: **Cruza** el **individuo actual** con el **individuo** de **entrada** de acuerdo con el tipo de operación de cruce.
* ``mutation(self)``: **Muta** el **individuo** de acuerdo con el tipo de operación de mutación.
* ``evaluate(self, problem)``: **Evalua** el **individuo** usando el **problema** a **resolver**.

Y los **métodos recomendados** son:

* ``__str__(self)``: **Representación** en formato de **cadena** de **caracteres** de un **individuo**. Método útil para depurar una lista de individuos.
* ``__repr__(self)``: **Método** invocado cuando se ejecuta **``print``** sobre el **individuo**. Método útil para depurar un solo individuo.

In [28]:
# PARA PROBAR LAS PRUEBAS DE CADA MÉTODO HAY QUE EJECUTAR PRIMERO LA CELDA FINAL DE EJECUCIÓN O DESCOMENTAR ESTA CELDA
with open("/data/notebook_files/example.json", 'r', encoding='utf8') as file:
   problemJSON = json.load(file)

In [29]:
class Individual:

    # =============================================================================
    # Constructor
    # =============================================================================

    # TODO: Code here the constructor
    def __init__ (self, num_genes, generation_type, crossover_type, mutation_type):
        self.num_genes = num_genes
        self.generation_type = generation_type
        self.crossover_type = crossover_type
        self.mutation_type = mutation_type
        self.IndSolution = self.generate(num_genes,generation_type)
        self.fitness = math.inf


    # =============================================================================
    # Mandatory methods
    # =============================================================================

    # TODO: Code here the mandatory methods

    #Método para generar individuos, se tiene en cuenta que solo existe un tipo de generación, aleatorio.
    #Al ser un método estático puede ser llamado desde fuera sin necesidad de crear un Individuo, además no podemos usar self
    @staticmethod
    def generate(num_genes, generation_type):
        if generation_type == "random":
            return list(np.random.permutation(num_genes))
        
        else:
            print("Tipo de generación no válido para permutaciones")


    #Método para aplicar cruce a dos Individuos padre, uno mismo y otro pasado por parámetro. 
    #El único cruce válido en permutaciones es el 2PCX
    def crossover(self, individual):
        aux1, aux2 = [], []
        hijo1 = Individual(self.num_genes,self.generation_type,self.crossover_type,self.mutation_type)
        hijo2 = Individual(self.num_genes,self.generation_type,self.crossover_type,self.mutation_type)

        if self.crossover_type == "2PCX":
            
            finish = False

            #Mientras finish sea False generamos números aleatorios para las posiciones de cruce, siempre Pos1 < Pos2
            while not finish:

                randomPosition1, randomPosition2 = random.randint(1, self.num_genes), random.randint(1, self.num_genes)
                if randomPosition1 < randomPosition2:
                    finish = True
            ###print("PCruce1: " + str(randomPosition1) + "   PCruce2: " + str(randomPosition2)+"\n")
                
            #Ya que en el constructor de Individual self.IndSolution coge los valores que le devuelve la función generate,
            #debemos vaciarlo antes de comenzar para tener dos vectores vacios de tamaño num_genes
            for i in range(self.num_genes):
                hijo1.IndSolution.pop(0)
                hijo2.IndSolution.pop(0)

            #Parte Izquierda [<=randomPosition1]
            for i in range(randomPosition1):
                hijo1.IndSolution.append(self.IndSolution[i])
                hijo2.IndSolution.append(individual.IndSolution[i])

            #Parte central [randomPosition+1,randomPosition2]
            for i in range(randomPosition1, randomPosition2):
                aux1.append(self.IndSolution[i])
                aux2.append(individual.IndSolution[i])

            #Insertamos en hijo1 con el orden de padre2
            for i in range(self.num_genes):
                if individual.IndSolution[i] in aux1:
                    hijo1.IndSolution.append(individual.IndSolution[i])

            #Insertamos en hijo2 con el orden de padre1
            for i in range(self.num_genes):
                if self.IndSolution[i] in aux2:
                    hijo2.IndSolution.append(self.IndSolution[i])

            #Parte derecha (>randomPosition2]
            for i in range(randomPosition2, self.num_genes):
                hijo1.IndSolution.append(self.IndSolution[i])
                hijo2.IndSolution.append(individual.IndSolution[i])

            return hijo1, hijo2

        else:
            print("Cruce no válido para permutaciones")

    
    #Método que nos permite mutar el individuo propio, tenemos dos tipos de mutaciones en permutaciones: Insertion y Swapping
    def mutation(self):
        finish = False
        mutado = Individual(self.num_genes,self.generation_type,self.crossover_type,self.mutation_type)
        
        #Al igual que antes construimos un bucle para obtener dos valores aleatorios de posición en los que Pos1 < Pos2
        while not finish:
            randomPosition1, randomPosition2 = random.randint(0, self.num_genes-1), random.randint(0, self.num_genes-1)
            if randomPosition1 < randomPosition2:
                finish = True
        
        ###print("Pos1: " + str(randomPosition1) + "   Pos2: " + str(randomPosition2)+"\n")
        value1, value2 = self.IndSolution[randomPosition1], self.IndSolution[randomPosition2]

        #En el caso de insertion cogemos el valor de la posición2 y lo insertamos después del valor de la posición1
        if self.mutation_type == "insertion":
            self.IndSolution.insert(randomPosition1+1, value2)
            self.IndSolution.pop(randomPosition2+1)

            mutado.IndSolution = self.IndSolution
            return mutado

        #En el caso de swapping intercambiamos los valores de posición, insertamos en la pos nueva y borramos en la pos vieja
        elif self.mutation_type == "swapping":
            self.IndSolution.insert(randomPosition1, value2)
            self.IndSolution.pop(randomPosition1+1)
            self.IndSolution.insert(randomPosition2, value1)
            self.IndSolution.pop(randomPosition2+1)

            mutado.IndSolution = self.IndSolution
            return mutado
        else:
            print("Tipo de mutación no implementado")
    

    #Este método evalua un individuo, para ello empleamos búsqueda en caminos para ir de una ciudad inicial a una ciudad objetivo
    #donde se encuentra el siguiente paquete a recoger. También tiene en cuenta la capacidad del camión, haciendo que vuelva al
    #almacén a descargar
    def evaluate(self, problem):
        totalCost = 0
        warehouse, estadoInicial = problem.warehouse, problem.warehouse
        vehicleCapacity = problem.maxCapacity


        #Recorremos el vector solución del Individuo aplicando el algoritmo de búsqueda en caminos
        #Empleamos el método search de CVRP para aplicar el algoritmo y, en caso de repetición de camino, leerá directamente
        #el resultado de caché para itar tiempo y computación de resultados
        i = 0
        while i < len(self.IndSolution):
            #Primero viajamos al siguiente objetivo haciendo la llamada a search desde nuestro inicio a nuestro objetivo
            estadoNext = problem.dicc[self.IndSolution[i]][0]
            totalCost += cvrp.search(estadoInicial, estadoNext)
            estadoInicial = estadoNext
            
            #Una vez llegamos comprobamos si el paquete ubicado en esa ciudad cabe en nuestro camión.
            #Si no cabe el camión debe volver desde esa ciudad (nuevo inicio) al almacén (warehouse) 
            if vehicleCapacity < problem.dicc[self.IndSolution[i]][1]:
                totalCost += cvrp.search(problem.dicc[self.IndSolution[i]][0], warehouse) 
                vehicleCapacity = problem.maxCapacity
                estadoInicial = warehouse

            #Si cabe lo metemos al camión y descontamos a la capacidad el peso del paquete y finalmente hacemos avanzar el
            #bucle para ir a por el siguiente paquete
            else:
                vehicleCapacity -=  problem.dicc[self.IndSolution[i]][1]
                i += 1

        #Una vez recogidos todos los paquetes debemos volver a nuestro almacén para hacer la última descarga de paquetes
        totalCost += cvrp.search(estadoInicial,warehouse)
        
        #Consideramos el fitness de un individuo el coste total en km del viaje.
        self.fitness = round(totalCost,2)
        return self.fitness

    # =============================================================================
    # Recommended methods
    # =============================================================================
    
    # TODO: Code here the recommended methods

    def __str__(self):
        return f"{self.IndSolution}"
    
    def __repr__(self):
        repr = ""
        for i in range(len(self.IndSolution)):
            repr = repr + str(self.IndSolution[i])
        return repr

**Se recomienda que se prueben cada uno de los métodos implementados de manera individual en las siguientes líneas de código:**

In [30]:
# TODO: Test here the methods to generate a solution
# individuo1 = Individual(len(problemJSON['parcels']), "random","", "")
# print(individuo1)

In [31]:
# TODO: Test here the methods to cross individuals
# individuo1 = Individual(len(problemJSON['parcels']), "random","2PCX", "")
# print("Padre 1: " + str(individuo1))
# individuo2 = Individual(len(problemJSON['parcels']), "random","2PCX", "")
# print("Padre 2: " + str(individuo2) + "\n")
# hijo1, hijo2 = individuo1.crossover(individuo2)
# print("\nHijo 1: "+ str(hijo1))
# print("Hijo 2: "+ str(hijo2))

In [32]:
# TODO: Test here the methods to mutate an individual
# individuo1 = Individual(len(problemJSON['parcels']), "random","2PCX", "swapping")
# print(individuo1)
# individuo1.mutation()
# print(individuo1)

#### Clase `Genetic`

Esta clase implementa un **esquema** básico de **algoritmo genético**.

Los **métodos obligatorios** que se deben añadir son:

* ``def __init__(self, population_size, num_generations, selection_type, crossover_type, crossover_probability, mutation_type, mutation_probability, keep_elitism, random_state)``: Inicializa el **tamaño** de la **población**, el **tipo** de **operación** de **selección**, **cruce**, y **mutación**, así como la **probabilidad** de aplicar las operaciones de **cruce** y **mutación**. Además, también inicializa el **número** de **mejores soluciones** de la **población actual** que se **mantienen** en la **siguiente población** y una **semilla** para garantizar que los **experimentos** son **reproducibles**. **Nótese que puede ser necesario añadir más argumentos si así se requiere**.
* ``def __call__(self, problem)``: **Método** que se **ejecuta** cuando se llama a un **objeto** de la **clase como** si fuese una **función**. En este **método** se debe **implementar** el **esquema básico** de un **algoritmo genético** que se encargue de ejecutar los pasos correspondientes. 
* ``def generate_population(self, problem)``: **Genera** la **población inicial** de acuerdo con el **problema** a resolver.
* ``def select_population(self, population, scores)``: **Selecciona** los **padres** a utilizar para la operación de cruce.
* ``def crossover(self, population)``: **Cruza pares** de **padres** teniendo en cuenta la probabilidad de cruce.
* ``def mutation(self, population)``: **Muta** los **individuos cruzados** teniendo en cuenta la probabilidad de mutación.
* ``def evaluate(self, population, problem)``: **Evalua** los **nuevos individuos** de acuerdo con el problema a resolver.
* ``def combine(self, population)``: **Forma** la **nueva generación** de acuerdo con el número de mejores individuos de la población actual a mantener en la siguiente.

In [33]:
class Genetic:

    # =============================================================================
    # Constructor
    # =============================================================================

    # TODO: Code here the constructor

    def __init__(self, population_size, num_generations, selection_type, crossover_type, crossover_probability, mutation_type, mutation_probability, keep_elitism, random_state):
        self.population_size = population_size
        self.num_generations = num_generations
        self.selection_type = selection_type
        self.crossover_type = crossover_type
        self.crossover_probability = crossover_probability
        self.mutation_type = mutation_type
        self.mutation_probability = mutation_probability
        self.keep_elitism = keep_elitism
        self.random_state = random_state
        self.evaluatedInd = []
        self.evaluatedNewInd = []
        self.pop = []
        self.evaluateddict = dict()



    # =============================================================================
    # Mandatory methods
    # =============================================================================

    # TODO: Code here the mandatory methods

    #Este método llama a todas las funciones posteriores siguiendo el esquema de un algoritmo genético y nos muestra las
    #diferentes poblaciones resultantes de cada uno de los procesos a partir de la población actual en cada generación.
    def __call__(self, problem):

        random.seed(self.random_state)
        
        #Inicializamos nuestra población
        pop = self.generate_population(problem)
        for i in range(self.num_generations):
            #print("---------------Generación "+str(i+1)+"---------------")
            ###print("----Current Pop----")
            ###for i in pop:
            ###    print(i)
            ###print("\n")

            #Evaluamos nuestra población actual
            self.selfevaluate(problem)

            #Seleccionamos individuos de nuestra población actual
            ###print("-------Selection-------")
            selected_pop = self.select_population(pop, self.evaluatedInd)
            ###for i in selected_pop:
            ###    print(i)
            ###print("\n")

            #Cruzamos los individuos seleccionados
            ###print("-------Crossover-------")
            cross_pop = self.crossover(selected_pop)
            ###print("\n")
            ###for i in cross_pop:
            ###    print(i)
            ###print("\n")

            #Mutamos los individuos seleccionados y cruzados
            ###print("-------Mutation-------")
            mutated_pop = self.mutation(cross_pop)
            ###print("\n")
            ###for i in mutated_pop:
            ###    print(i)
            ###print("\n")

            #Finalmente evaluamos la población resultante y la combinamos con la actual
            self.evaluate(mutated_pop, problem)
             
            ###print("-------Combine-------")
            pop = self.combine(mutated_pop)
            ###for i in pop:
            ###    print(i)
            ###print("\n")

        return pop

    
    #Generamos los individuos de la población, tantos como nos permita el population_size
    #Además añadimos valores a dos arrays, con los que trabajaremos luego, para que tengan tamaño population_size y
    #evitar problemas con valores fuera de index posteriormente.
    def generate_population(self, problem):
        ngenes = len(problem.dicc)
        
        for i in range(self.population_size):
            individuo = Individual(ngenes,"random",self.crossover_type, self.mutation_type)
            self.pop.append(individuo)
            self.evaluatedInd.append(0.0)
            self.evaluatedNewInd.append(0.0)
        return self.pop
    

    #Método para seleccionar individuos de la población actual, en función del fitness del individuo
    #Nuestro objetivo es muestrear con reemplazo una distribución de probabilidad
    def select_population(self, population, scores):
        fitness_denom = 0.0
        probability = 0.0
        prob_fitness = []
        selected_pop = []

        #En nuestro caso queremos minimizar el fitness puesto que queremos minimizar el número de km, por ello la fórmula cambia
        #a (1/x)/sum(1/xi)
        if self.selection_type == "fitness":
            
            #Primero calculamos la suma del denominador

            for i in scores:
                fitness_denom += 1/i 
            
            #Finalmente calculamos la probabilidad para cada individuo en función de su fitness, a cada individuo se le suma
            #la probabilidad del individuo anterior.
            for i in population:

                probability += (1/scores[population.index(i)])/fitness_denom
                prob_fitness.append(probability)

            #Para seleccionar debemos generar números aleatorios del 0 al 1 y añadiendo a la población seleccionada aquel 
            #primer individuo que tenga una probabilidad acumulativa mayor que el número random.
            for i in range(self.population_size):
                randomNumber = random.uniform(0, 1)

                for j in range(len(prob_fitness)):
                    if randomNumber < prob_fitness[j]:
                        selected_pop.append(population[j])
                        break

            return selected_pop

        elif self.selection_type == "range":
            print("Método de selección no implementado")

        elif self.selection_type == "tournament": 
            print("Método de selección no implementado")

        else:
            print("Método de selección inválido")
    

    #Método para realizar el cruce entre los individuos de las poblaciones, para ello debemos conocer el número de parejas
    #dividiendo entre 2, en caso de que exista resto (número de población impar), este individuo se añade al final
    def crossover(self, population):
        
        if self.crossover_type == "2PCX":
        
            newPop = []
    
            num_pairs = int(len(population)/2)
            aux = 0

            #Recorriendo las parejas sacamos los padres (es necesario hacer un deepcopy ya que sin el, al modificar parent1 y 2,
            #también se modificaría population[aux])
            for i in range(num_pairs):
                parent1 = copy.deepcopy(population[aux])
                aux += 1
                parent2 = copy.deepcopy(population[aux])
                aux += 1

                #Generamos números aleatorios y si el número es menor de la probabilidad de cruce se produce el cruce llamando
                #al método crossover de la clase Individual, si se produce cruce añadimos los dos hijos y si no los padres
                randomNumber = random.uniform(0, 1)
                if randomNumber < self.crossover_probability:
                    ###print("Pareja: "+str(i+1)+"    ")
                    child1, child2 = parent1.crossover(parent2)
                    newPop.append(child1)
                    newPop.append(child2)
                else:
                    newPop.append(parent1)
                    newPop.append(parent2)
        
            if len(population) % 2 != 0:
                newPop.append(copy.deepcopy(population[len(population) - 1]))

            return newPop
        else:
            print("Cruce no válido para permutaciones")
    

    #Método para realizar la mutación de los individuos de la población.
    def mutation(self, population):
        mutatedPop = []
        aux=1

        for i in population:
            #Para cada individuo de la población generamos su número random, si es menor que la probabilidad de mutación
            #se produce la mutación llamando al método mutation de la clase Individual.
            
            randomNumber = random.uniform(0, 1)
            if randomNumber < self.mutation_probability:
                ###print("Cromosoma: "+str(aux)+"    ")
                mutatedPop.append(copy.deepcopy(i.mutation()))
            else:
                mutatedPop.append(copy.deepcopy(i))
            aux += 1
        return mutatedPop
    

    #Método adicional, creado simplemente para el método combinar, que emplea dos poblaciones, evalua todos los individuos
    #de la población actual, para ello llama al método evaluate de Individual
    def selfevaluate(self, problem):

        for i in range(self.population_size):
            if str(self.pop[i].IndSolution) in self.evaluateddict:
                self.evaluatedInd[i] = self.evaluateddict[str(self.pop[i].IndSolution)]
            else:
                self.evaluatedInd[i] = round(self.pop[i].evaluate(problem),2)
                self.evaluateddict[str(self.pop[i].IndSolution)] = self.evaluatedInd[i]



    #Método para evaluar los individuos de la población pasada por parámetro, para ello llama al método evaluate de Individual
    def evaluate(self, population, problem):

        for i in range(len(population)):
            if str(population[i].IndSolution) in self.evaluateddict:
                self.evaluatedNewInd[i] = self.evaluateddict[str(population[i].IndSolution)]          
            else:
                self.evaluatedNewInd[i] = round(population[i].evaluate(problem),2)
                self.evaluateddict[str(population[i].IndSolution)] = self.evaluatedNewInd[i]

            

    #Método que nos permite combinar dos poblaciones, la actual y aquella seleccionada después de aplicar cruce y mutación
    def combine(self, population):
        finalPop = []
        #Almacenamos el vector de evaluación de la población actual
        evaluated = copy.deepcopy(self.evaluatedInd)
        #Almacenamos el vector de evaluación de la población seleccionada, cruzada y mutada
        evaluatedNew = copy.deepcopy(self.evaluatedNewInd)

        rest = self.population_size
        #Si keep-elitism es True, debemos añadir a la población resultado el individuo de menor fitness de la población actual
        for i in range(self.keep_elitism):
            valorMin = math.inf
            #Sacamos el valor mínimo
            for j in evaluated:
                if j < valorMin:
                    valorMin = j

            #Borramos el individuo de valor mínimo, en caso de haber dos se borra solo el primero (break)
            for j in evaluated:
                if evaluated:
                    if j == valorMin:
                        evaluated.remove(j)
                        break
            
            #Añadimos ese individuo a la población final
            for j in range(len(self.evaluatedInd)):   
                if self.evaluatedInd[j] == valorMin:
                    finalPop.append(self.pop[j])
                    break
            
            rest -= 1
    

        #Para el tamaño de población restante hacemos el mismo proceso para la población seleccionada, cruzada y mutada
        for i in range(rest):
            valorMin = math.inf
                
            for i in evaluatedNew:
                if i < valorMin:
                    valorMin = i

            for i in evaluatedNew:
                if evaluatedNew:
                    if i == valorMin:
                        evaluatedNew.remove(i)
                        break

            for i in range(len(self.evaluatedNewInd)):
                if self.evaluatedNewInd[i] == valorMin:
                    finalPop.append(population[i])  
                    break

        for i in range(self.population_size):
            self.pop[i]=finalPop[i]

        return finalPop
            

**Se recomienda que se prueben cada uno de los métodos implementados de manera individual en las siguientes líneas de código:**

In [34]:
# TODO: Test here the methods to generate a population
# genetico = Genetic(10, 1, "fitness", "2PCX", 1, "swapping", 1, False, 123)
# cvrp = CVRP("/data/notebook_files/example.json",genetico)
# pop = genetico.generate_population(cvrp)
# print("Current Population")
# for i in pop:
#     print(i)
# print("\n")

In [35]:
# TODO: Test here the methods to select individuals
# cvrp = CVRP("/data/notebook_files/example.json",genetico)
# genetico.selfevaluate(cvrp)
# selectedPop = genetico.select_population(pop,genetico.evaluatedInd)
# for i in selectedPop:
#     print(i)

In [36]:
#Test here the methods to crossover populations
# print("Selected Population")
# for i in selectedPop:
#     print(i)
# print("\n")

# newPop = genetico.crossover(selectedPop)

# print("\n")
# print("Crossover Population")
# for i in newPop:
#     print(i)

In [37]:
#Test here the methods to mutate populations
# print("Crossover Population")
# for i in newPop:
#     print(i)
# print("\n")

# mutatedPop = genetico.mutation(newPop)

# print("\n")
# print("Crossover+Mutated Population")
# for i in mutatedPop:
#     print(i)

In [38]:
#Comparación de la población con los individuos seleccionados tras cruce y mutación
# cvrp = CVRP("/data/notebook_files/example.json",genetico)

# print("Current Population")
# for i in pop:
#     print(i)
# print("\n")
# genetico.evaluate(pop, cvrp)
# print(genetico.evaluatedInd)
# print("------------------------------------------------------")
# genetico.evaluate(mutatedPop, cvrp)
# print(genetico.evaluatedNewInd)
# print("\n")
# print("Crossover+Mutated Population")
# for i in mutatedPop:
#     print(i)

In [39]:
# TODO: Test here the methods to combine populations
# print("Combined/Final Population")
# finalPop = genetico.combine(mutatedPop)
# for i in finalPop:
#     print(i)

#### Clase `CVRP`

Esta clase representa el problema en cuestión, esto es, el **problema** de **ruteo** de **vehículos**.

Los **métodos obligatorios** que se deben añadir son:

* ``def __init__(self, filename, algorithm)``: Inicializa el **problema** en cuestión y el **algoritmo** a usar para resolverlo. A su vez, se debe crear un **diccionario** que contenga como **clave** un **identificador** de **paquete** y como **valor** una **tupla** con la **ciudad** donde se encuentra dicho paquete y su **peso**.
* ``def __call__(self)``: **Resuelve** el **problema** en cuestión.
* ``def evaluate(self, solution)``: **Evalua** una **solución** para el **problema** en cuestión, **teniendo en cuenta** las **restricciones correspondientes**.
* ``def search(self, departure, goal)``: **Resuelve** un **problema** de **búsqueda** de **caminos** dada las **ciudades** de **salida** y **meta**.

**Nótese que se puede crear una estructura de datos para agilizar el proceso de búsqueda de caminos requerido por el algoritmo ¿cuál puede ser?**

In [40]:
# TODO: Introduce here the answer to use for the hashing
answer = "cache"

# Avoid case sensitivity in the answer
answer = str.lower(answer)

# Encode the answer before hashing
answer = answer.encode("utf-8")

encoded = "0fea6a13c52b4d4725368f24b045ca84"  # The hashed answer
print(encoded)

check_answer(answer, encoded)

0fea6a13c52b4d4725368f24b045ca84


'The answer is correct.'

In [41]:
class CVRP:

    # =============================================================================
    # Constructor
    # =============================================================================

    # TODO: Code here the constructor
    
    def __init__(self, filename, algorithm):
        with open(filename, 'r', encoding='utf8') as file:
            problemJSON = json.load(file)
        
        self.prob_class = utils.Problem(problemJSON)
        self.algorithm = algorithm
        self.solution = []
        self.dicc = {parcel['id']: (parcel['city'],parcel['weight']) for parcel in problemJSON['parcels']}
        self.maxCapacity = problemJSON['vehicles'][0]['capacity']
        self.warehouse = problemJSON['warehouse']
        self.cache = dict()
        self.usosAStar = 0
        self.usosCache = 0

    # =============================================================================
    # Mandatory methods
    # =============================================================================

    # TODO: Code here the mandatory methods

    #Con este método llamamos al algoritmo Genético pasándole el problema por parámetro y evaluamos la solución final
    def __call__(self):
        valormin = math.inf
        tiempoInicio = time.perf_counter()
        self.solution = self.algorithm(self)
        evaluated = self.evaluate(self.solution)
        print("Puntuación media: "+str(np.mean(evaluated)))
        for i in evaluated:
            if i < valormin:
                valormin = i
        for i in evaluated:
            if i == valormin:
                print("La puntuación del mejor individuo es: " +str(i))
                break
        
        tiempoFinal = time.perf_counter()
        tiempoTotal = round((tiempoFinal - tiempoInicio)*1000,4)
        print("Tiempo de ejecución: " + str(tiempoTotal) + " milisegundos")
        print("Usos de Caché: "+str(self.usosCache))
        print("Usos de AStar: "+str(self.usosAStar))
        return self.solution
    
    #Este método evalua la solución pasada como parámetro
    def evaluate(self, solution):
        self.algorithm.evaluate(solution, self.dicc)
        #print("\nEvaluación Final: \n" +str(self.algorithm.evaluatedNewInd))
        return self.algorithm.evaluatedNewInd

    #Este método nos resuelve el problema de búsqueda de caminos, para ello emplea AStar, siempre con la heurística euclidea,
    #además inplementa una caché que nos servirá para ahorrar tiempo y computación, para ello primero comprueba si el camino 
    #está guardado en caché, si esta lo devuelve y si no llama a AStar y guarda el resultado en caché.
    def search(self, departure, goal):

        self.prob_class.initial_state = utils.State(departure)
        self.prob_class.final_city = utils.State(goal)
       
        if str((departure,goal)) in self.cache:
            #print("Dentro de cache")
            self.usosCache += 1
            return self.cache[str((departure,goal))]
            
        else:
            #print("Fuera de cache")
            search = utils.AStar(self.prob_class, 'euclidea')
            self.usosAStar += 1
            cost = round(search.doSearch(),2)
            self.cache[str((departure,goal))], self.cache[str((goal,departure))] = cost, cost
        return cost

**Se recomienda que se prueben cada uno de los métodos implementados de manera individual en las siguientes líneas de código:**

In [42]:
# TODO: Test here the method to initialize the capacited vehicle routing problem
#genetico = Genetic(10, 100, "fitness", "2PCX", 0.9, "swapping", 0.1, 5, 0)
#cvrp = CVRP("/data/notebook_files/example.json",genetico)

In [43]:
# TODO: Test here the method to solve the search problem
#Para comprobar si el valor está dentro o fuera de caché hay que descomentar los prints del método search
#cvrp.search(3,0)

In [44]:
# TODO: Test here the method to solve the capacited vehicle routing problem
# Para ejecutar esta tercera celda primero hay que ejecutar primero la primera (2 arriba) cada vez
#sol = cvrp()
#print("Solución Final:")
#for i in sol:
#    print(i)

### 4.3. Estudio y mejora de los algoritmos

Una vez que los algoritmos han sido implementados, se debe **estudiar** su **rendimiento**. Para ello, se debe comparar la **calidad** de las **soluciones obtenidas**, así como las **diferentes estadísticas** que se consideren adecuadas (número de generaciones, tiempo de ejecución, etc.). Factores como el tamaño máximo del problema que se soporta sin un error de memoria, así como el efecto temporal de usar escenarios más complejos son otros factores a tener en cuenta. Además, se **pueden proponer** y se valorarán la incorporación de **técnicas** que **permitan acelerar** la **ejecución** de los **algoritmos**.

---

In [45]:
#Celda de Ejecución utilizada para el apartado de Evaluación de la Memoria
PopSize = 50
NumGens = 100
SelectionType = "fitness"
CrossType = "2PCX"
CrossProb = 0.8
MutType = "insertion" #También puede tomar valor "insertion"
MutProb = 0.8
Elitism = 5
Seed = 1



#Pasamos por parámetro el JSON empleado en CVRP
print("example------------------")
genetico = Genetic(PopSize, NumGens, SelectionType, CrossType, CrossProb, MutType, MutProb, Elitism, Seed)
cvrp = CVRP("/data/notebook_files/example.json",genetico)
sol1 = cvrp()
print("small--------------------")
genetico = Genetic(PopSize, NumGens, SelectionType, CrossType, CrossProb, MutType, MutProb, Elitism, Seed)
cvrp = CVRP("/data/notebook_files/small.json",genetico)
sol2 = cvrp()
print("medium-------------------")
genetico = Genetic(PopSize, NumGens, SelectionType, CrossType, CrossProb, MutType, MutProb, Elitism, Seed)
cvrp = CVRP("/data/notebook_files/medium.json",genetico)
sol3 = cvrp()
print("large--------------------")
genetico = Genetic(PopSize, NumGens, SelectionType, CrossType, CrossProb, MutType, MutProb, Elitism, Seed)
cvrp = CVRP("/data/notebook_files/large.json",genetico)
sol4 = cvrp()
print("parcels3----------------")
genetico = Genetic(PopSize, NumGens, SelectionType, CrossType, CrossProb, MutType, MutProb, Elitism, Seed)
cvrp = CVRP("/data/notebook_files/parcels3.json",genetico)
sol5 = cvrp()

In [46]:
# TODO: Experiment here with the small problem
#Abrimos el fichero json con el que trabajaremos
# Para ver que está ocurriendo en cada generación hay que desmarcar todos los comentarios ###, se pueden encontrar
# facilmente con ctrl+f y buscando '###'
# PopSize = 50
# NumGens = 100
# SelectionType = "fitness"
# CrossType = "2PCX"
# CrossProb = 0.9
# MutType = "swapping" #También puede tomar valor "insertion"
# MutProb = 0.1
# Elitism = False
# Seed = 12345

# genetico = Genetic(PopSize, NumGens, SelectionType, CrossType, CrossProb, MutType, MutProb, Elitism, Seed)

# #Pasamos por parámetro el JSON empleado en CVRP
# cvrp = CVRP("/data/notebook_files/small.json",genetico)
# sol = cvrp()
# print("Solución Final:")
# for i in sol:
#     print(i)
    

In [47]:
# TODO: Experiment here with the medium problem
#Abrimos el fichero json con el que trabajaremos
# Para ver que está ocurriendo en cada generación hay que desmarcar todos los comentarios ###, se pueden encontrar
# facilmente con ctrl+f y buscando '###'
# PopSize = 50
# NumGens = 100
# SelectionType = "fitness"
# CrossType = "2PCX"
# CrossProb = 0.9
# MutType = "swapping" #También puede tomar valor "insertion"
# MutProb = 0.1
# Elitism = False
# Seed = 123

# genetico = Genetic(PopSize, NumGens, SelectionType, CrossType, CrossProb, MutType, MutProb, Elitism, Seed)

# #Pasamos por parámetro el JSON empleado en CVRP
# cvrp = CVRP("/data/notebook_files/medium.json",genetico)
# sol = cvrp()
# print("Solución Final:")
# for i in sol:
#     print(i)

In [48]:
# TODO: Experiment here with the large problem
#Abrimos el fichero json con el que trabajaremos
# Para ver que está ocurriendo en cada generación hay que desmarcar todos los comentarios ###, se pueden encontrar
# facilmente con ctrl+f y buscando '###'
# PopSize = 50
# NumGens = 100
# SelectionType = "fitness"
# CrossType = "2PCX"
# CrossProb = 0.9
# MutType = "swapping" #También puede tomar valor "insertion"
# MutProb = 0.1
# Elitism = False
# Seed = 1234

# genetico = Genetic(PopSize, NumGens, SelectionType, CrossType, CrossProb, MutType, MutProb, Elitism, Seed)

# #Pasamos por parámetro el JSON empleado en CVRP
# cvrp = CVRP("/data/notebook_files/large.json",genetico)
# sol = cvrp()
# print("Solución Final:")
# for i in sol:
#     print(i)

In [49]:
# TODO: Experiment here with the any problem
#Abrimos el fichero json con el que trabajaremos
# Para ver que está ocurriendo en cada generación hay que desmarcar todos los comentarios ###, se pueden encontrar
# facilmente con ctrl+f y buscando '###'
# PopSize = 50
# NumGens = 100
# SelectionType = "fitness"
# CrossType = "2PCX"
# CrossProb = 0.9
# MutType = "swapping" #También puede tomar valor "insertion"
# MutProb = 0.1
# Elitism = False
# Seed = 123

# genetico = Genetic(PopSize, NumGens, SelectionType, CrossType, CrossProb, MutType, MutProb, Elitism, Seed)

# #Pasamos por parámetro el JSON empleado en CVRP
# cvrp = CVRP("/data/notebook_files/parcels3.json",genetico)
# sol = cvrp()
# print("Solución Final:")
# for i in sol:
#     print(i)

Puntuación media: 210108.7518
La puntuación del mejor individuo es: 195367.13
Tiempo de ejecución: 55626.9857 milisegundos
Usos de Caché: 840758
Usos de AStar: 554
Solución Final:
[7, 72, 52, 71, 58, 57, 77, 61, 78, 76, 39, 8, 37, 49, 48, 2, 62, 28, 69, 70, 64, 16, 15, 34, 68, 24, 32, 55, 63, 12, 36, 53, 19, 45, 66, 33, 20, 59, 3, 0, 67, 74, 5, 79, 47, 60, 11, 65, 42, 18, 40, 41, 29, 23, 51, 35, 38, 13, 22, 1, 4, 75, 54, 73, 10, 31, 14, 27, 56, 43, 44, 26, 50, 21, 46, 30, 6, 9, 25, 17]
[7, 57, 77, 58, 72, 61, 52, 78, 11, 36, 39, 71, 8, 37, 49, 76, 48, 2, 34, 62, 28, 69, 70, 32, 24, 64, 16, 15, 41, 68, 0, 53, 79, 12, 63, 19, 33, 45, 5, 20, 51, 67, 47, 3, 66, 74, 65, 40, 42, 23, 60, 13, 29, 18, 55, 59, 35, 38, 27, 22, 1, 4, 75, 54, 31, 73, 46, 44, 6, 43, 26, 50, 10, 14, 56, 21, 30, 9, 25, 17]
[7, 57, 77, 58, 72, 61, 52, 78, 11, 36, 39, 71, 8, 37, 49, 48, 2, 34, 62, 28, 69, 70, 68, 76, 64, 15, 0, 12, 16, 53, 20, 41, 32, 79, 24, 63, 19, 33, 45, 5, 51, 67, 47, 3, 66, 74, 65, 40, 42, 23, 60,

### 5. Entrega y evaluación

Al igual que la práctica anterior, esta se debe **hacer en pares**. No obstante, en **casos excepcionales** se permite realizarla **individualmente**. **La fecha límite para subir la práctica es el 18 de diciembre de 2022 a las 23:55**. Las **entrevistas y evaluaciones** se realizarán la **semana siguiente**.

Algunas consideraciones:

* **En caso de que no se haya entregado la primera práctica, o se haya sacado menos de un cuatro, se podrán entregar conjuntamente en esta fecha. No obstante, se considerará únicamente un 90% de la nota global de prácticas**.
* Está práctica supone el **70%** de la **nota** en este apartado.
* La práctica se **evaluará** mediante una **entrevista individual** con el profesorado. Las fechas de las entrevistas se publicarán con antelación.
*  Se proporcionará un **conjunto** de **casos** de **prueba preliminares** (varios mapas e instancias) que se **deben resolver correctamente**. En caso contrario, la práctica se considerará suspensa.
* La **entrevista** consistirá en una serie de **preguntas** acerca del **código**.

**Por último, para la evaluación no continua se requirirá la implementación del algoritmo de búsqueda por ascenso de colinas. Además, este se deberá utilizar para inicializar la población del algoritmo genético, en lugar de que sea aleatoria.**